In [44]:
import numpy as np
import pandas as pd
from random import randint
from statistics import mode
import random
random.seed(0)

In [45]:
df_pipo = pd.read_excel("C:/Users/arnav/Downloads/The-Track-Fitness-Club-master/The-Track-Fitness-Club-master/Clean Data/PIPO.xlsx")

In [46]:
df_invoice = pd.read_excel("C:/Users/arnav/Downloads/The-Track-Fitness-Club-master/The-Track-Fitness-Club-master/Clean Data/invoice data.xlsx")

In [47]:
df_pipo.head(2)

,#Rno,In Time,Out Time,Hour,Day,Month,Year,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,943,2022-01-01 16:19:36,2022-01-01 17:39:12,16,Saturday,1,2022,NaN,NaN,NaN,NaN
1,943,2022-01-03 17:23:00,2022-01-03 18:46:00,17,Monday,1,2022,NaN,NaN,NaN,NaN


In [48]:
df_invoice.head(2)

,Sr.no,#Rno,PaymentDate,package,validity,subscription,first_month,Month,Discount,Paid Amount,Package Text,Duration/Sessions,Revenue,#Rno.1,subscription.1
0,1,943,2022-01-01,3,2022-04-01,Expire,1,1,0,2700,3 month.,90/90,2700,943,Expire
1,2,953,2022-01-01,1,2022-01-31,Expire,2021,1,0,1000,1 month.,30/30,1000,953,Expire


In [49]:
df_invoice["subscription.1"][0]

'Expire'

In [50]:
# make a list of unique clients
client_unique = []  # [1018,1019,1020,1021,1023,1024,...] 870 unique clients
for i in range(len(df_invoice)):
    if df_invoice["#Rno"][i] not in client_unique:
        client_unique.append(df_invoice["#Rno"][i])

print("Total number of Unique Clients in the Data is :", len(client_unique))
print("Registeration numbers of clients:",client_unique[0:5],"...")

Total number of Unique Clients in the Data is : 494
Registeration numbers of clients: [943, 953, 1018, 1019, 1020] ...


In [51]:
# view the first 10 unique clients

client_unique[0:10]

[943, 953, 1018, 1019, 1020, 1021, 1024, 1025, 1026, 1027]

In [52]:
# EDeclare the gym slot variables to store the dedicated hours for each slot

mor_slot = [6,7,8,9,10]
noon_slot = [11,12,13,14,15]
eve_slot = [16,17]
late_eve_slot = [18,19,20,21]

# STEPS to find noon and evening clients who are active and flexible:
### a. client : [list of all the slots he ever visited for workouts ] --> dict {client slot dict}        
### b. client : preferred slot --> dict  {client preferred slot dict}
### c. client : %deviation from preferred slot -- >dict    {client slot jump value dict}
### d. flexible client : preferred slot of thee flexible clients --> dict    
### e. active fleexible client : preferred slot of these active flexible clients -- > dict

## A

In [53]:
clients_slot_dict = {}  
# key will be Rno and value will be a list consisting of all workout hours of the client, then the workout hour will be converted
# into timing slots and then the slot which appears maximum will label the client in that slot

In [54]:
for c in client_unique:
    client_hour_list = [] # refresh the list back to null every time the client is changed
    for i in range(len(df_pipo)-1):
        if df_pipo["#Rno"][i] == c:
            if df_pipo["Hour"][i] in mor_slot:
                client_hour_list.append("morning slot")
            elif df_pipo["Hour"][i] in noon_slot:
                client_hour_list.append("noon slot")
            elif df_pipo["Hour"][i] in eve_slot:
                client_hour_list.append("evening slot")
            elif df_pipo["Hour"][i] in late_eve_slot:
                client_hour_list.append("late evening slot")
                
    clients_slot_dict[c] = client_hour_list

In [55]:
# print daywise preferred workout slots of the first 10 clients
print("The initial five clients from the dataset, along with all their daywise workout time slots\n")
for client, preferred_slot in list(clients_slot_dict.items())[:5]:
    print(f'Client {client} preferred {preferred_slot}')



The initial five clients from the dataset, along with all their daywise workout time slots

Client 943 preferred ['evening slot', 'evening slot', 'evening slot', 'late evening slot', 'late evening slot', 'evening slot', 'morning slot', 'late evening slot', 'morning slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'morning slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'morning slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'evening slot', 'morning slot', 'evening slot', 'late evening slot', 'evening slot', 'late evening slot', 'evening slot

## B. Find the most "preferred" slot of the all the unique clients in the history of 8 months, most preferred slot is the slot where a client belongs to 

In [56]:
clients_preferred_slot_dict = {} # keys will be Rno and values will be %of the preferred slot or %of most occuring slot

In [57]:
for k,v in clients_slot_dict.items():  # k is Rno and v is a list consisting of all hours this client workedout in the history of his duration
    clients_preferred_slot_dict[k] = mode(v)  # mode(list) = returns the most occuring value in the list

In [58]:
print("The initial five clients from the dataset, along with their preferred workout time slots\n")
for client, most_preferred_slot in list(clients_preferred_slot_dict.items())[:5]:
    print(f"Client {client} mostly prefers {most_preferred_slot}")



The initial five clients from the dataset, along with their preferred workout time slots

Client 943 mostly prefers evening slot
Client 953 mostly prefers noon slot
Client 1018 mostly prefers late evening slot
Client 1019 mostly prefers morning slot
Client 1020 mostly prefers morning slot


## C 
## now we have clients and their preferred slot we can find how many times a client deviates from its preferred slot

In [59]:
client_slot_jump_value_dict = {}  #len = 494
for unique_client in client_unique:
    jump = 0  # reset jump everytime a new client appears
    workout_count = 0 # reset workout count everytime a new client appears
    for i in range(len(df_pipo)-1):
        if df_pipo["#Rno"][i] == unique_client:
            workout_count += 1          
            
            preferred_client_slot_is = clients_preferred_slot_dict[unique_client]  # know the clients preferred slot
            
            if preferred_client_slot_is == "morning slot":
                preferred_client_slot_is = mor_slot  # mor_slot = [6,7,8,9,10]
                
                if df_pipo["Hour"][i] not in preferred_client_slot_is:         
                        jump += 1
            
            elif preferred_client_slot_is == "noon slot":
                preferred_client_slot_is = noon_slot
                
                if df_pipo["Hour"][i] not in preferred_client_slot_is:         
                        jump += 1
            
            elif preferred_client_slot_is == "evening slot":
                preferred_client_slot_is = eve_slot
                
                if df_pipo["Hour"][i] not in preferred_client_slot_is:         
                        jump += 1
            elif preferred_client_slot_is == "late evening slot":
                preferred_client_slot_is = late_eve_slot
                
                if df_pipo["Hour"][i] not in preferred_client_slot_is:         
                        jump += 1
    
    client_slot_jump_value_dict[unique_client] = round((jump/workout_count)*100,2)
    
          

In [60]:
print(len(client_slot_jump_value_dict))

494


In [61]:
print("The initial five clients from the dataset, along with their deviation from their most preferred time slots\n")

for i, (client, deviation) in enumerate(client_slot_jump_value_dict.items()):
    print(f"Client {client} deviated from their most preferred workout slot {deviation}% of times")
    if i == 4:
        break


The initial five clients from the dataset, along with their deviation from their most preferred time slots

Client 943 deviated from their most preferred workout slot 19.81% of times
Client 953 deviated from their most preferred workout slot 32.65% of times
Client 1018 deviated from their most preferred workout slot 0.0% of times
Client 1019 deviated from their most preferred workout slot 9.34% of times
Client 1020 deviated from their most preferred workout slot 5.49% of times


# d. Flexible clients

In [62]:
total_flexible_clients_list = []  # len = 173, tolerance goes up, flexibility goes down
tolerance = 25
for k,v in client_slot_jump_value_dict.items():
    if v > tolerance:
        total_flexible_clients_list.append(k)

print(" no.of Flexible clients throughout history :",len(total_flexible_clients_list))

 no.of Flexible clients throughout history : 97


In [63]:
print(total_flexible_clients_list)

[953, 1044, 1049, 1052, 1053, 1057, 983, 1073, 1076, 1081, 1090, 989, 1094, 1101, 1102, 971, 1107, 1109, 1120, 1121, 1124, 768, 860, 944, 1160, 1164, 863, 772, 774, 876, 986, 868, 1225, 1229, 1233, 993, 1008, 786, 789, 793, 1246, 1328, 794, 1358, 1265, 1383, 798, 922, 1391, 854, 803, 804, 1333, 1334, 805, 806, 1272, 1371, 1378, 919, 811, 813, 1376, 879, 880, 1248, 886, 1418, 1495, 1529, 851, 881, 823, 824, 1405, 1629, 1493, 1633, 1648, 1115, 1503, 1657, 833, 1685, 1707, 1731, 1738, 1565, 1581, 1704, 1714, 1414, 1589, 1808, 1836, 869, 1574]


In [64]:
#lets see the preferred slots of all the 95 flexible clients
all_flexible_c_preferred_slot_dict = {}  # create an empty dictionary to later append necessary keys and values
for afc in total_flexible_clients_list:
    for c,slot in clients_preferred_slot_dict.items():
        if afc == c:  # if a client is flexible
            all_flexible_c_preferred_slot_dict[afc] = slot
        
print(len(all_flexible_c_preferred_slot_dict))

97


In [65]:
print(all_flexible_c_preferred_slot_dict)  # total 95 flexible clients and their preferred slot

{953: 'noon slot', 1044: 'noon slot', 1049: 'noon slot', 1052: 'evening slot', 1053: 'evening slot', 1057: 'noon slot', 983: 'noon slot', 1073: 'noon slot', 1076: 'noon slot', 1081: 'noon slot', 1090: 'evening slot', 989: 'evening slot', 1094: 'noon slot', 1101: 'noon slot', 1102: 'noon slot', 971: 'evening slot', 1107: 'evening slot', 1109: 'noon slot', 1120: 'noon slot', 1121: 'evening slot', 1124: 'noon slot', 768: 'noon slot', 860: 'noon slot', 944: 'evening slot', 1160: 'evening slot', 1164: 'noon slot', 863: 'noon slot', 772: 'evening slot', 774: 'evening slot', 876: 'morning slot', 986: 'evening slot', 868: 'noon slot', 1225: 'noon slot', 1229: 'noon slot', 1233: 'noon slot', 993: 'evening slot', 1008: 'noon slot', 786: 'noon slot', 789: 'noon slot', 793: 'noon slot', 1246: 'evening slot', 1328: 'noon slot', 794: 'evening slot', 1358: 'noon slot', 1265: 'noon slot', 1383: 'noon slot', 798: 'noon slot', 922: 'evening slot', 1391: 'evening slot', 854: 'noon slot', 803: 'evening sl

## E. Extract Active flexible clients from all 95 flexible clients

In [66]:
active_flexible_clients_dict = {}
for k,v in all_flexible_c_preferred_slot_dict.items(): # tracing a dict of all flexible clients with their preferred slot    
    for i in range(len(df_invoice)): 
        if df_invoice["#Rno"][i] == k and df_invoice["subscription.1"][i] == "Active":
            if k in total_flexible_clients_list:  # if the client is the one we are looking for
                active_flexible_clients_dict[k] = v # extract noonevening active clients from the total 95 clients
                break

In [67]:
print(len(active_flexible_clients_dict))  # total 36 active and flexible clients

38


In [68]:
active_flexible_noonEveningPreferring_clients_dict = {}
for k,v in active_flexible_clients_dict.items():
    if v == "noon slot" or v == "evening slot":
        active_flexible_noonEveningPreferring_clients_dict[k] = v
print(len(active_flexible_noonEveningPreferring_clients_dict))  # all active clients flexible clients perfer noon evening slots slots

37


In [69]:
df_active_flexible_noonEveningPreferring_clients = pd.DataFrame(active_flexible_noonEveningPreferring_clients_dict.items(), columns= ["#Rno", "preferred slot"])

In [70]:
df_active_flexible_noonEveningPreferring_clients

,#Rno,preferred slot
0,1044,noon slot
1,1052,evening slot
2,1057,noon slot
3,1094,noon slot
4,1109,noon slot
5,1124,noon slot
6,863,noon slot
7,868,noon slot
8,1328,noon slot
9,794,evening slot


In [71]:
# df_active_flexible_noonEveningPreferring_clients.to_excel("active flexible clients.xlsx", index = False)